<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Introduction to CSV.jl and DataFrames.jl</p><br>
<p style="font-family: Arial; font-size:2.00em;color:green; font-style:bold">
Kumar Rahul</p>

**_DataFrames_** is a Julia library for tabular data manipulation. It offers a number of data exploration, cleaning and transformation operations that are critical in working with data in Julia. Similar to pandas in Python and data.table, dplyr in R. DataFrames.jl work well with a range of file formats such as CSVs (using CSV.jl), Apache Arrow (using Arrow.jl) Stata, SPSS, and SAS files (using StatFiles.jl), and reading and writing parquet files (using Parquet.jl)


Objects of the DataFrame type represent a data table as a series of vectors, each corresponding to a column or variable.  After a brief introduction to these two data structures and data ingestion, the key features of *dataframes* this notebook covers are:
>* Generating descriptive statistics on data
* Data cleaning using built in dataframes functions
* Frequent data operations for subsetting, filtering, insertion, deletion and aggregation of data
* Merging multiple datasets using dataframes

Few other libraries which people may have to use with DataFrames for advanced data wrangling:

> * Impute.jl: various methods for handling missing data in vectors, matrices and tables.
* DataFramesMeta.jl: A range of convenience functions for DataFrames.jl that augment select and transform to provide a user experience similar to that provided by dplyr in R.
* Query.jl: Query.jl provides a single framework for data wrangling that works with a range of libraries, including DataFrames.jl, other tabular data libraries (more on those below), and even non-tabular data. Provides many convenience functions analogous to those in dplyr in R or LINQ. SQL like querying structure.
* StatsModels.jl: For converting heterogeneous DataFrame into homogenous matrices for use with linear algebra libraries or machine learning applications that don't directly support DataFrames. Will do things like convert categorical variables into indicators/one-hot-encodings, create interaction terms, etc.

**Additional Recommended Resources:**
* `DataFrames` documentation: https://dataframes.juliadata.org/stable/
* `DataFrames Cheatsheet`: https://www.ahsmart.com/pub/data-wrangling-with-data-frames-jl-cheat-sheet/
* Comparing with R & Python: https://dataframes.juliadata.org/stable/man/comparisons/

## Importing library

In [ ]:
#using Pkg
#Pkg.add("DataFrames") ##Mandatory
#Pkg.add("CSV") ##Mandatory
#Pkg.add("Statistics") ## Mandatory - For statistics like mean, quaretile etc.

#Pkg.add("FreqTables") ## Good to have for Pivoting
#Pkg.add("StatsBase") ### Good to have - counting, ranking, covariances, sampling, and empirical density estimation.

#Pkg.add("TableView") ## optional - to render a table
#Pkg.add("DataFramesMeta") ##optional - Data manipulation add-on to DataFrames.jl. SQL style statements - Query.jl 42 or DataFramesMeta.jl

#Pkg.add("WebIO") ##Not needed.

In [ ]:
using CSV
using DataFrames
using Statistics
using FreqTables
using StatsBase
using DataFramesMeta

In [ ]:
#using WebIO
#WebIO.install_jupyter_nbextension()
#using TableView

## General settings 

By default, Julia uses 80 horizontal space to show columns and 30 vertical space to show rows.
> * Check the default number of rows and columns which are displayed. 
* Change the default setting.

In [ ]:
ENV["COLUMNS"]

In [ ]:
ENV["LINES"]

There is no analogus of the below python command here:

    > from IPython.core.interactiveshell import InteractiveShell
    > InteractiveShell.ast_node_interactivity = "all"
    
To display all objects from a code cell, use as below:

In [ ]:
a = 2;
println(a);
b = 4;
@show b;

use of colon, is optional, and it suppresses the default print from the code. 

## Create a dataframe

Several ways to create a dataframe. One being shown below:

In [ ]:
df = DataFrame()
df.A = 1:8
df.B = ["M", "F", "F", "M", "F", "M", "M", "F"]
df

# Case - Titanic Dataset

The data has been taken from : https://www.kaggle.com/c/titanic/data

The data description:


> * PassengerId: type should be integers
* Survived: Survived or Not
* Pclass: Class of Travel
* Name: Name of Passenger
* Sex: Gender
* Age: Age of Passengers
* SibSp: Number of Sibling/Spouse aboard
* Parch: Number of Parent/Child aboard
* Ticket
* Fare
* Cabin
* Embarked: The port in which a passenger has embarked. C - Cherbourg, S - Southampton, Q = Queenstown


We will use the titanic dataset to understand data load and munging using DataFrames.

In [ ]:
#os.getcwd()
homedir()

In [ ]:
pwd()

In [ ]:
cd("../")

In [ ]:
pwd()

In [ ]:
cd("./Julia_Practice")

In [ ]:
pwd()

## Read the dataset, which is in csv format

Pandas has many read_* functions to read data from multiples data sources or formats like json, jdbc, excel, pickel (python serialized objects) etc.

In [ ]:
ENV["COLUMNS"] = 1000

In [ ]:
#?CSV.File

In [ ]:
#titanic_df = pd.csv("./data/titanic.csv")


titanic_df = CSV.File("./data/titanic.csv") |> DataFrame
#titanic_df = CSV.read("./data/titanic.csv", DataFrame)

For reading file which is not in UTF-8 encoding (Say, ISO-8859-1), we may need StringEncodings.jl. 

In [ ]:
typeof(titanic_df)

## Print the first few rows
head() also takes an argument n, which specifies how many records will be printed.

In [ ]:
#titanic.head()

## To return only the first record
first(titanic_df);
head(titanic_df)


In [ ]:
# titanic.tail(10)

tail(titanic_df,10);
last(titanic_df)

## Statistical Summary

Use `describe` method

In [ ]:
#?describe

In [ ]:
#titanic_df.describe()

describe(titanic_df, :all)

> **Get descriptive for Age and Fare column.**

In [ ]:
describe(titanic_df, :all, cols=[:Age,:Fare])

> **Get descriptive for Sex and Embarked column.**

In [ ]:
describe(titanic_df, :all, cols=["Sex","Embarked"])

## Indexing and Slicing - With Copy

The df[:, :col] and df[:, cols] syntaxes always copy dataframes so they are safe to use (and should generally be preferred except for performance or memory critical use cases). You can also use copy(df). Below code use this syntax:
* Select all rows across all columns.

In [ ]:
#titanic_df.iloc[:,:]

titanic_df[:,:]

> **Select 5 rows across first 3 columns.**

In [ ]:
#titanic_df.iloc[0:5,0:3]

titanic_df[1:5,1:3]

> **Select first 4 rows and first 3 columns.**

In [ ]:
#titanic_df.iloc[:4,:3]

titanic_df[1:4,1:3]

> **Select first 2 columns of the last row.**

In [ ]:
#titanic_df.iloc[-1:,:2]

titanic_df[end,1:2]

> **Last 3 records can be selected this way.**

In [ ]:
titanic_df[end-2:end,:]

> **Select first 3 records.**

In [ ]:
titanic_df[1:3,:]

> **First 2 columns not selected.**

In [ ]:
head(titanic_df[:, Not(1:2)])

> **First row not selected.**

In [ ]:
head(titanic_df[Not(1),:])

> **Select columns between two column names.**

In [ ]:
head(titanic_df[:, Between(:Pclass, :Age)])

> **Select the record of column whose name starts with "A".**

In [ ]:
head(titanic_df[:, r"A"])

> **Select the record of column whose name starts with "A" but do not select the first record.**

In [ ]:
head(titanic_df[Not(1), r"A"])

`r` stands for Regex. One can write advanced Regex. 

## Indexing and Slicing - Without Copy

**Caution**
With df[!, :col] and df.col syntax you get a direct (non copying) access to a column of a data frame. This is potentially unsafe as you can easily corrupt data in the df data frame if you resize, sort, etc. the column obtained in this way. Therefore such access should be used with caution.

Similarly df[!, cols] when cols is a collection of columns produces a new data frame that holds the same (not copied) columns as the source df data frame. Similarly, modifying the data frame obtained via df[!, cols] might cause problems with the consistency of df.

! indicates that underlying columns are not copied. Ex: Select the record of column whose name starts with "A".

In [ ]:
head(titanic_df[!, r"A"])

In [ ]:
head(titanic_df[!, Not(1)])

## Working with Views

Create a view of a DataFrame which is more efficient than creating a selection. Here are the possible return value options. Exactly the same syntax as slicing and indexing as above but for the `@view` prefixed.

In [ ]:
head(@view (titanic_df[:, r"A"]))

In [ ]:
@view titanic_df[1:4,1:3]

## Check the data type

> * Data type
* dimensions.. how many row and columns
* Structure of the data

> **Dimensions of data.**

In [ ]:
#?size

In [ ]:
#titanic_df.shape
@show size(titanic_df)
@show size(titanic_df, 1)
@show size(titanic_df, 2)

In [ ]:
nrow(titanic_df), ncol(titanic_df)

What are the columns and their types. Each column has how many not-null values. `eltype` means element types.

In [ ]:
#titanic_df.info()
eltypes(titanic_df)

In [ ]:
eltype.(eachcol(titanic_df))

> **Column names of titanic_df.**

In [ ]:
names(titanic_df)

> **Column names as symbol.**

In [ ]:
propertynames(titanic_df)

DataFrames.jl allows to use Symbols (like :A) and strings (like "A") for all column indexing operations for convenience. However, using Symbols is slightly faster and should generally be preferred, if not generating them via string manipulation.

## Select Specific columns and rows¶

> **Select the Survived column and display the first 5 entries.**

In [ ]:
#titanic_df[['Survived']][0:5]

@view titanic_df[["Survived"]][1:5,1]

In [ ]:
@view titanic_df[["Survived"]][1:5,1]

In [ ]:
#titanic_df[['Survived']].head(5)

head(@view titanic_df[["Survived"]],5)

In [ ]:
#The other way
#titanic_df.Survived[0:5]
    
@view titanic_df.Survived[1:5,1]

Get the vector stored in the DataFrame without copying it. The `.` and `!` (stands for inplace) both can achieve this:

In [ ]:
titanic_df."Survived"

titanic_df.Survived

titanic_df[!, ["Survived"]]

head(titanic_df[!, [:Survived]])

In [ ]:
typeof(titanic_df.Survived[1:5,1])

The syntax below will creates a copy (not advisable)

In [ ]:
head(titanic_df[:, ["Survived"]])

## Selecting multiple columns. 

Multiple column names should be provided as a list

In [ ]:
#titanic_df[['Survived','Age']].head()
#titanic_df[['Survived','Age']][1:5]
#titanic_df[['Survived','Age']].iloc[0:5,]

head(@view titanic_df[:,["Survived","Age"]])
@view titanic_df[:,["Survived","Age"]][1:6,:]
@view titanic_df[:,[:Survived,:Age]][1:6,:]

Select a column using the `select()` function from DataFrames pacakage.

In [ ]:
head(select(titanic_df,[:Survived,:Age]))

## Condition based filters

This will work:

In [ ]:
size(titanic_df[(titanic_df.Pclass .== 1), :])

This will work as well:

In [ ]:
size(titanic_df[(titanic_df.Pclass .== 1) .& (titanic_df.Sex .=="male"), :])

> **How many children below 5 years age were on board the ship**

But this won't as there are missing values in this column.

In [ ]:
titanic_df[titanic_df.Age .<= 5.0, :]

If there is missing value in a column, then condition filter gives error. In such cases, use coalesce. It return the first value in the arguments which is not equal to missing, if any.

In [ ]:
#new_df = titanic_df[(titanic_df.Age <=5) ]

size(titanic_df[coalesce.(titanic_df.Age .<= 5.0, false), :])

> **Filter with just missing value in Age column.**

In [ ]:
size(titanic_df[(ismissing.(titanic_df.Age)), :])

> **Filter with missing value and age <=5 years.**

In [ ]:
size(titanic_df[(coalesce.(titanic_df.Age .<= 5.0, true)), :])

> **How many children survived who are less than 5 years old**

In [ ]:
#titanic_df [(titanic_df.Age <=5) & (titanic_df.Survived ==1)].shape

size(titanic_df[coalesce.(titanic_df.Age .<=5, false) .& (titanic_df.Survived .==1),:])

In [ ]:
#titanic_df[(titanic_df.Age <= 5)].shape

countmap((titanic_df[coalesce.(titanic_df.Age .<=5, false),:]).Survived);

countmap((titanic_df[coalesce.(titanic_df.Age .<=5, false) .& (titanic_df.Survived .==1),:]).Survived)

> **How many people survived and what is the percentage.**

In [ ]:
#titanic_df[ titanic_df.Age <= 5].Survived.value_counts(normalize = True)

proptable((titanic_df[coalesce.(titanic_df.Age .<=5, false),:]).Survived)

> **Select Age, Sex and Pclass of passengers who survived and where less than 5 years old**

In [ ]:
#titanic_df[ (titanic_df.Age <= 5) &  
#            (titanic_df.Survived ==1) ][['Age','Gender','Pclass']].head()

head(titanic_df[coalesce.(titanic_df.Age, false) .<=5 .& 
                    (titanic_df.Survived .==1),[:Age, :Sex,:Pclass]])

> **Only age, sex, survived and pclass of passengers whose age are not known.**

In [ ]:
#titanic_df[ titanic_df.Age.isnull() ][['Age','Gender','Pclass']].shape

head(titanic_df[ismissing.(titanic_df.Age),[:Age, :Sex,:Pclass]])

> **Only age, sex, survived and pclass of passengers whose age are known.**

In [ ]:
#titanic_df[ -titanic_df.Age.isnull() ][['Age','Gender','Pclass']][0:5]

head(titanic_df[Not(ismissing.(titanic_df.Age)),[:Age, :Sex,:Pclass]])

> **Several other ways of conditon based filter**

In [ ]:
@which findall

In [ ]:
filter(row -> row.Sex=="male", titanic_df);

filter(:Sex => ==("male") , titanic_df);

titanic_df[titanic_df.Sex .== "male", :];

titanic_df[findall(==("male"), titanic_df.Sex), :];

titanic_df[isequal.(titanic_df.Pclass,2), :];

titanic_df[findall(==(1), titanic_df.Pclass), :];

titanic_df[findall(<=(1), titanic_df.Pclass), :];

## Get unique values for a column

> **How many embark points were there.**

In [ ]:
#titanic_df.Embarked.unique()

unique(titanic_df.Embarked)

> **How many missing value in Embarked column.**

In [ ]:
countmap(titanic_df.Embarked)

## Cross tabulation of data

> **Pivot gender with survived.**

In [ ]:
DataFrames.combine(DataFrames.groupby(titanic_df,[:Sex,:Survived]), nrow => :count)

In [ ]:
#pd.crosstab(titanic_df.Sex, titanic_df.Survived, margins = True)

FreqTbl= FreqTables.freqtable(titanic_df, :Sex, :Survived)

In [ ]:
typeof(FreqTbl)

In [ ]:
convert(DataFrame,FreqTbl)

In [ ]:
rename(convert(DataFrame,FreqTbl), Dict(:x1=>string(0),:x2=>string(1)))

### Normalize by Rows

If margins is 1 row proportions are calculated

In [ ]:
#pd.crosstab(titanic_df.Sex,titanic_df.Survived, margins=True, normalize='index')

FreqTables.proptable(titanic_df,:Sex,:Survived, margins =1)

### Normalize by columns

If margins is 2 column proportions are calculated

In [ ]:
#pd.crosstab(titanic_df.Sex,titanic_df.Survived, margins=True, normalize='columns')
FreqTables.proptable(titanic_df,:Sex,:Survived, margins =2)

### Normalize by row and column

In [ ]:
#pd.crosstab(titanic_df.Sex,titanic_df.Survived, margins=True,normalize='all')

FreqTables.proptable(titanic_df,:Sex,:Survived)

## Aggregation 

Getting summary statistics for a column can be done by using functions from DataFrames, StatsBase and FreqTable package.


To know which package a function belogs to: use @which as a prefix. Say, you want to know which package the function combine belongs to:

In [ ]:
@which combine

In [ ]:
## Using statistics package
DataFrames.combine(DataFrames.groupby(titanic_df,[:Survived]), nrow => :count)

In [ ]:
@which countmap

In [ ]:
#titanic_df.Survived.value_counts()

## using statsbase package
StatsBase.countmap(titanic_df.Survived)
#StatsBase.counts(titanic_df.Survived)

In [ ]:
#titanic_df.Survived.value_counts(normalize = True)

StatsBase.proportionmap(titanic_df.Survived)

In [ ]:
#The other way
#titanic_df['Survived'].value_counts()

StatsBase.proportionmap(titanic_df["Survived"])

In [ ]:
## using FreqTable package. Useful in Pivots as well. will see later
FreqTables.freqtable(titanic_df.Survived, skipmissing=true)

In [ ]:
freqtable(titanic_df,:Survived)

## Working with NA Values

ismissing just asks if the column itself is a missing type, which it is not.

In [ ]:
ismissing(titanic_df)

In [ ]:
any(ismissing.(titanic_df.Age))

#all(ismissing.(titanic_df.Age))

How many missing:

In [ ]:
sum(ismissing.(titanic_df.Age))

Use replace!() if you want to repalce the missing value in the same column of a dataframe.

In [ ]:
replace(titanic_df.Age, missing=>1);

The other way wherein we want to replace the missing value with some value, say 0 as well as apply more than one math operation:

In [ ]:
@which ismissing

In [ ]:
map(x -> ismissing(x) ? 0.0 : [log(x),sqrt(x)], titanic_df.Age)

Change the value of of a selection using broadcasting.

In [ ]:
titanic_df[1:2,[:Age,:SibSp]]

The symbol `.` is for broadcasting.

In [ ]:
titanic_df[1:2,[:Age,:SibSp]] .= [25,2]

## Drop NA: Strategy

In [ ]:
@which dropmissing

> **Drop NA Values in Embarked column.**

In [ ]:
#titanic_df.Embarked.dropna(inplace=False)

size(dropmissing(titanic_df,:Embarked))

> **Remove rows where there are NA values in any of the columns.**

In [ ]:
#clean_titanic_df = titanic_df.dropna()

clean_titanic_df = dropmissing(titanic_df)
size(clean_titanic_df)

> **Look for missing in only cabin and delete the entire record if missing is found.**

In [ ]:
head(dropmissing(titanic_df, :Cabin),2)

In [ ]:
size(dropmissing(titanic_df, [:Cabin,:Sex]))

## Rename a Column

> **Rename "Embarked" column to "onboarded"**

In [ ]:
#titanic_df = titanic_df.rename(columns = {"Embarked": "Onboarded", "Sex": "Gender"}, inplace=False)

head(rename(titanic_df, :Embarked=> :Onboarded, :Sex=>:Gender),2)

In [ ]:
head(rename(titanic_df, [:Embarked=> :Onboarded, :Sex=>:Gender]),2)

In [ ]:
head(rename(titanic_df, Dict(:Embarked=> :Onboarded, :Sex=>:Gender)),2)

In [ ]:
head(rename(uppercase,titanic_df),2)

**To make the change to the original dataframe, use as rename!(). rename!() is similar to inplace parameter in pandas.**

## Groupby and Aggregate

> **Find the average age of passengers based on Pclass column.**

In [ ]:
# groupby+combine at the same time
sort(@by(
    titanic_df,[:Pclass], 
    first=first(:Age), 
    last=last(:Age),
    mean=mean(skipmissing(:Age))
    )) 

In [ ]:
grouped_df = groupby(titanic_df,[:Pclass]);

In [ ]:
# the same as by but on grouped DataFrame

sort(@combine(grouped_df, 
        first=first(:Age), 
        last=last(:Age), 
        mean=mean(skipmissing(:Age))
    ))

> **Compute several statistics on the grouped dataframe.** 

By using the combine function from DataFrames.jl

In [ ]:
sort(combine(grouped_df, :Age .=> [first, last, mean]))

> **Find the average age of passengers based on Pclass and Gender.**

In [ ]:
##Python code
#pclass_gender_age_df = titanic_df.groupby(['Pclass', 'Gender'])['Age'].mean().reset_index()

In [ ]:
##Using DataFramemeta package
pclass_gender_age_df = @combine(groupby(titanic_df,[:Pclass,:Sex]),Avg=mean(skipmissing(:Age)))

The other way, we can create a new dataframe with completecases for Pclass, Sex, Age:

In [ ]:
subset_complete_df = titanic_df[completecases(titanic_df[:,[:Pclass,:Sex,:Age]]),:]
size(subset_complete_df)

In [ ]:
##using DataFrame package
pclass_gender_age_df = combine(groupby(subset_complete_df,[:Pclass,:Sex]),:Age.=>[mean,first,last,std])

> **How many passengers survived in each of the pclass and for each of the gender.**

In [ ]:
#pclass_gender_sur_df = titanic_df.groupby( ['Pclass', "Gender"] )["Survived"].sum().reset_index()


pclass_gender_sur_df = @combine(groupby(titanic_df,[:Pclass,:Sex]),Survived=sum(skipmissing(:Survived)))


## Merge Dataframe

> Merge pclass_gender_age_df with pclass_gender_sur_df based on pclass and Gender

In [ ]:
#pclass_gender_merge_df = pclass_gender_age_df.merge(pclass_gender_sur_df, on = ['Pclass','Gender'])

pclass_gender_merge_df = innerjoin(pclass_gender_age_df,pclass_gender_sur_df,on = ["Pclass","Sex"])

## Sort Values

> Sort the pclass_gender_merge_df on Survived column

In [ ]:
#pclass_gender_merge_df.sort_values("Survived", ascending=False)

sort(pclass_gender_merge_df,[:Pclass,:Survived],rev=[false, false])

In [ ]:
#pclass_gender_merge_df.to_csv("merged_df.csv")

CSV.write("merged_df.csv", pclass_gender_merge_df)

### Exercise: 

Using `pclass_gender_merge_df` do a visualization of Age and Survived w.r.t Pclass and Sex. What kind of charts will be meaningful here

## Get the categorical and numeric features

This is how we can do in Python

In [ ]:
#import numpy as np

#num_feat_list = [x for x in titanic_df.select_dtypes(include = np.number)]
#num_feat_list

#cat_feat_list = [x for x in titanic_df.select_dtypes(include = np.object)]
#cat_feat_list


In [ ]:
#Base.nonmissingtype

Create a list of categorical and numeric features in a dataset in Julia. The `<:` operator in general means "is a subtype of", and used in declarations as: 

* used in expressions as a subtype operator which returns true when its left operand is a subtype of its right operand
* declares the right-hand type to be an immediate supertype of the newly declared type. 

In [ ]:
#num_feat_list = names(titanic_df, String)

num_feat_list = names(titanic_df[[i for i in names(titanic_df) if nonmissingtype(eltype(titanic_df[i])) <: Number]])

#num_feat_list = names(titanic_df[(<:).(eltypes(titanic_df),Union{Number,Missing})])


In [ ]:
#cat_feat_list = names(titanic_df[[i for i in names(titanic_df) if Base.nonmissingtype(eltype(titanic_df[i])) <: String]])
cat_feat_list = names(titanic_df[(<:).(eltypes(titanic_df),Union{String,Missing})])


Benefit is evident while summarizing and performing any other operation on features of the same type:

In [ ]:
describe(titanic_df, :all, cols=num_feat_list)

## Adding new column

> **Create a vector of ones which has the same size as length of titanic_df**

In [ ]:
@which ones

In [ ]:
AllOnes = Base.ones(size(titanic_df,1));

> **Add the vector of ones to titanic_df.**

In [ ]:
titanic_df[:, "Add_Ones"] = AllOnes; #Creates a copy
head(titanic_df,2)

> **Check if the All_ones is a copy of the vector.** 

isequal, isless, and === produce results of type Bool.

In [ ]:
titanic_df.Add_Ones === AllOnes

False means that new memory was used and Add_ones is not refering to the vector of ones.

In [ ]:
titanic_df[!, "Add_Ones_Again"] = AllOnes; #Without creating a copy
head(titanic_df,2)

In [ ]:
titanic_df.Add_Ones_Again === AllOnes

> **Add column named 'ImptAge' by replacing the missing values in 'Age' column with '1'.**

In [ ]:
head(insertcols!(titanic_df, 7, :ImptAge => replace(titanic_df.Age, missing=>1),makeunique=true))

> **Adding a feature in the dataframe which is log of Age column.**

In [ ]:
titanic_df[!, "LogAge1"] = log.(titanic_df.Age)
head(titanic_df)

> **Adding an index column in titanic_df.**

In [ ]:
insertcols!(titanic_df, 1, :Sl_No => 1:size(titanic_df,1),makeunique=true)
head(titanic_df,2)


hcat(), vcat() for horizontal or vertical concatnation.

### Add column - Lambda function with map

Math operations to create new features can be referred: https://docs.julialang.org/en/v1/manual/mathematical-operations/. In case of missing value in a column, most math functions handle it. So we may compute as below:

In [ ]:
@which sqrt

In [ ]:
map(x -> [sqrt(x)], titanic_df[:,:Age])[1:6,]

Broadcasting with `.` operator after the function name also gives the same output. We will use broadcating to add columns later.

In [ ]:
sqrt.(titanic_df.Age)[1:6]

> **Transforming the new column 'ImptAge' by applying log function.**

In [ ]:
titanic_df.logAge2 = map(x -> log(x), titanic_df[:,:ImptAge])
head(titanic_df,2)

> **Create a Column named Gender and map Sex Column.**

In [ ]:
#titanic_df['gender_code'] = titanic_df.Gender.map(lambda x: int(x =='male'))
#titanic_df.iloc[0:4,]

##One way
titanic_df.gender_code1= map(x -> (x=="male") ? 1 : 0, titanic_df[:,:Sex])


##The other way
insertcols!(titanic_df, 7, :gender_code2 => map(x -> (x=="male") ? 1 : 0, titanic_df[:,:Sex]),makeunique=true);


head(titanic_df,2)

> **Adding a feature in the dataframe which is square root of Age column.**

In [ ]:
insertcols!(titanic_df, 7, :SqrtAge => map(x -> sqrt(x), titanic_df[:,:Age]),makeunique=true);

### Add columns by broadcasting

> **Adding Column of ones and zeroes by broadcasting.**

The '.' operator stands for broadcast.

In [ ]:
titanic_df[:, "OneByBroadcast"] .= 1;
titanic_df[!, "ZeoresByBroadcast"] .= 0;
head(titanic_df,2)

> **Adding a feature in the dataframe which is square root of Age column.**

In [ ]:
## We did this with lambda and map earlier. The other way
insertcols!(titanic_df, 7, :SqrtAge =>sqrt.(titanic_df.Age),makeunique=true);

head(titanic_df)

## User defined functions


> **Define a function to find NA values and unique labels in any column.**

In [ ]:
#def find_na(name):
#    return titanic_df[name].value_counts(dropna=False)

function find_na(name)
    return countmap(titanic_df[:,name])
end

In [ ]:
#def find_unique(name):
#    return titanic_df[name].unique()

function find_unique(name)
    return unique(titanic_df[:,name])
end

In [ ]:
find_na("Embarked")

In [ ]:
find_unique("Embarked")

> **Define a function to perform cross tabulation.**

In [ ]:
#def cross_tab(x,y):
#    return pd.crosstab(titanic_df[x],titanic_df[y])

function cross_tab(x,y)
    return freqtable(titanic_df[:,x],titanic_df[:,y])
end

In [ ]:
ct = (cross_tab("Sex","Pclass"))
ct

Converting to a dataframe

In [ ]:
## In Python
#stacked_df = df.stack().reset_index().rename(columns={0:'percentage'})
#stacked_df

In [ ]:
rn=names(ct,1);
cn=names(ct,2);
df=DataFrame(ct, Symbol.(cn))
insertcols!(df, 1, :rowname => rn,makeunique=true)

## This can also be done
#df.rowname = rn

> **Converting a dataframe from wide to long format.**

In [ ]:
stack_df = stack(df, ["1","2","3"],"rowname", variable_name = "Pclass",value_name ="count") 

> **Visualizing cross tabulated data created above.**

In [ ]:
#import seaborn as sn
#sn.barplot(x='Pclass', y='percentage', hue='Gender', data = stacked_df)

In [ ]:
#Pkg.add("Gadfly")

In [ ]:
using Gadfly

In [ ]:
set_default_plot_size(20cm, 8cm)

In [ ]:
bp1 = Gadfly.plot(stack_df, 
    x="Pclass",
    y="count", 
    Geom.bar(),
    Theme(bar_spacing=1mm, key_position=:none),
    Scale.color_discrete_manual("red")
)

bp2 = Gadfly.plot(stack_df, 
    x=:Pclass,
    y=:count, 
    color=:rowname, 
    Geom.bar(position=:dodge),
    Theme(bar_spacing=1mm, key_position=:right),
    Scale.color_discrete_manual("red","green")
)

hstack(bp1,bp2)

## Exercise 1: 

1. Use the user defined function `find_na` and `find_unique` to get the value count and unique labels of all the categorical features in the titanic dataframe


The commented code are in python. Think of converting it to Julia code with the syntaxes learnt in this notebook.

In [ ]:
#import numpy as np

#cat_feat_list = [x for x in titanic_df.select_dtypes(include = np.object)]
#cat_feat_list

In [ ]:
#cat_feat_list.append("Survived")
#cat_feat_list.remove('Name')
#cat_feat_list.remove('Ticket')
#cat_feat_list.remove('Cabin')
#cat_feat_list

In [ ]:
#for c in cat_feat_list:
#    find_na(c)
#    find_unique(c)

Other way:

In [ ]:
#print(list(map(find_na, cat_feat_list)))
#print(list(map(find_unique, cat_feat_list)))

## Exercise 2:

2. Use the user defined function `cross_tab` to cross tabulate every pair of categorical features. 

The commented code are in python. Think of converting it to Julia code with the syntaxes learnt in this notebook.

#### First Method

Cross tabulation of every pair of categorical feature.

In [ ]:
#for x in range(0,len(cat_feat_list)):
#    for y in range(x+1,len(cat_feat_list)):
#        #if x!=y:
#            cross_tab(cat_feat_list[x],cat_feat_list[y])

#### Second Method

In [ ]:
#import itertools
#for p in itertools.combinations(cat_feat_list, 2):
#    cross_tab(*p)

## Exercise 3: DIY
3. Change the cross_tab function to report the nomalized values by rows and columns. Perform visulization for each pair of categorical variable. What kind of charts will be meaningful here

## Advanced Topics: Working with Other file formats

XML, JSON , BSON, YAML , MessagePack, and protobuf are some commonly used data serialization formats.

JDF is a serailization format supported by Julia. JDF stores a DataFrame in a folder with each column stored as a separate file. There is also a metadata.jls file that stores metadata about the original DataFrame. Collectively, the column files, the metadata file, and the folder is called a JDF "file".

JDF.jl is a pure-Julia solution and there are a lot of ways to do nifty things like compression and encapsulating the underlying struture of the arrays that's hard to do in R and Python. E.g. Python's numpy arrays are C objects, but all the vector types used in JDF are Julia data types.

JDF is a DataFrames serialization format with the following goals

* Fast save and load times
* Compressed storage on disk
* Enable disk-based data manipulation (not yet achieved; from v0.4.0)
* Supports machine learning workloads, e.g. mini-batch, sampling (not yet achieved; from v0.4.0)


More here: https://github.com/xiaodaigh/JDF.jl

In [ ]:
using JDF

In [ ]:
#JDF.save("merged_df.jdf", pclass_gender_merge_df);

JDF.save("titanic.jdf", titanic_df)

**Note**

If the columns names in the dataframe have the same name, JDF.save will overwrite the saved file. Thus, while using JDF.load(), it may error out.

In [ ]:
titanic_jdf = JDF.load("titanic.jdf") |> DataFrame

> **Create an object which is a on disk representation of `titanic_df`.**

In [ ]:

titanic_ondisk = jdf"titanic.jdf"

In [ ]:
names(titanic_ondisk)

> **Load only Age and Pclass.**

In [ ]:
jdf_df = JDF.load(titanic_ondisk; cols = ["Age", "Pclass"]) |> DataFrame
head(jdf_df)

Other file formats:

* JSONTables.jl: To do read and write data in JSON format.
* Arrow.jl: Apache Arrow format that allows, in particular, for data interchange with R or Python.

## Stack and Unstack - Wide - Long - Wide format

pclass_gender_age_df  created earlier is in wide format.

In [ ]:
head(pclass_gender_age_df,2)

> **Select Age_mean and Age_Std to convert it into long format.**

Pass measure variables and then id-variable. Renaming variable is optional.

In [ ]:
stack_df = stack(pclass_gender_age_df, [:Age_mean,:Age_std], [:Pclass,:Sex],
    variable_name="key", value_name="observed") # 

> **Convert the stack_df created above into long format.**

Without key (the column name in above df) no unstacking can be done.

In [ ]:
unstack(stack_df, [:Pclass,:Sex], :key, :observed)

> **Wide format with all variable and no renaming of columns.**

In [ ]:
stack_df = stack(pclass_gender_age_df)

> **Convert the stack_df created above into long format.**

The name variable in above dataframe is the key

In [ ]:
unstack(stack_df, [:Pclass,:Sex], :variable, :value)

## Thank You